In [83]:
import pandas as pd

In [79]:
# Multiple Linear Regression

def MLR():

    from sklearn.linear_model import LinearRegression
    from sklearn.model_selection import train_test_split
    import statsmodels.formula.api as sm

    df = pd.read_csv("Master_Data_Set.csv")
    df = df.set_index(df["House"])
    del df["House"]
    print(df.head())
    print("-----------------------------------------------------------------------------")
    
    X = df.iloc[:,:-1].values
    y = df.iloc[:,-1].values
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=50)
    
    mlr = LinearRegression().fit(X_train, y_train)
    print("Score: ", mlr.score(X_test, y_test))
    print("Coefficients: ", mlr.coef_)
    print("Intercept: ", mlr.intercept_)
    y_pred = mlr.predict(X_test)
    
    model = sm.OLS(y, X)
    result = model.fit()
    print(result.summary())
    
    return 

In [80]:
# Lasso

def Lasso():
    
    from sklearn import linear_model
    from sklearn.model_selection import train_test_split
    import statsmodels.formula.api as sm

    df = pd.read_csv("Master_Data_Set.csv")
    df = df.set_index(df["House"])
    del df["House"]
    print(df.head())
    print("-----------------------------------------------------------------------------")
    
    X = df.iloc[:,:-1].values
    y = df.iloc[:,-1].values
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=50)
    
    lasso = linear_model.Lasso(alpha=0.1).fit(X_train, y_train)
    print("Score: ", lasso.score(X_test, y_test))
    print("Coefficients: ", lasso.coef_)
    print("Intercept: ", lasso.intercept_)
    print('Predictors with non-zero coefficients:', [i for i, item in enumerate(lasso.coef_) if abs(item) > 0])

    y_pred = lasso.predict(X_test)
    
    model = sm.OLS(endog=y, exog=X)
    result = model.fit()
    print(result.summary())
    
    return 


In [81]:
mlr = MLR()

         Duration     Peak  Volume  Outdoor_Events_x  Above_avg_vol  \
House                                                                 
12S1003     40340  1495.51  659.69                 0              0   
12S1008     38340  1019.95  539.90                 0              0   
12S1009     30420   602.32  842.53                 0              1   
12S1010     54120  1097.70  644.98                 0              0   
12S1011     47630  1095.79  771.40                 0              1   

         Above_avg_dur  Above_avg_peak  Outdoor_Events_y  Outdoor_User  
House                                                                   
12S1003              1               1                 0             0  
12S1008              1               1                 0             0  
12S1009              0               0                 0             0  
12S1010              1               1                 0             0  
12S1011              1               1                 0        

In [82]:
lasso = Lasso()

         Duration     Peak  Volume  Outdoor_Events_x  Above_avg_vol  \
House                                                                 
12S1003     40340  1495.51  659.69                 0              0   
12S1008     38340  1019.95  539.90                 0              0   
12S1009     30420   602.32  842.53                 0              1   
12S1010     54120  1097.70  644.98                 0              0   
12S1011     47630  1095.79  771.40                 0              1   

         Above_avg_dur  Above_avg_peak  Outdoor_Events_y  Outdoor_User  
House                                                                   
12S1003              1               1                 0             0  
12S1008              1               1                 0             0  
12S1009              0               0                 0             0  
12S1010              1               1                 0             0  
12S1011              1               1                 0        